In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
r"""
@DATE: 2024-11-15 17:22:03
@File: test_fuc/vdktest.ipynb
@IDE: vscode
@Description:
    VikingDB 连接测试
"""

'\n@DATE: 2024-11-15 17:22:03\n@File: test_fuc/vdktest.ipynb\n@IDE: vscode\n@Description:\n    VikingDB 连接测试\n'

In [ ]:
from volcengine.viking_db import *
#host中不需要加http或者https等前缀
#当使用http时，host可以填写ip或域名, 当使用https时, host只能填写域名
#scheme 可选，可选值：http/https，默认http。
#connection_timeout，可配置，默认30s。
#socket_timeout，可配置，默认30s。
vikingdb_service = VikingDBService(host="api-vikingdb.volces.com", region="cn-beijing",connection_timeout=100)


In [16]:
fields = [
    Field(
        field_name="doc_id",
        field_type=FieldType.String,
        is_primary_key=True
    ),
    Field(
        field_name="text_vector",
        field_type=FieldType.Vector,
        dim=12
    ),
    Field(
        field_name="text_sparse_vector",
        field_type=FieldType.Sparse_Vector
    ),
    Field(
        field_name="like",
        field_type=FieldType.Int64,
        default_val=0
    ),
    Field(
        field_name="price",
        field_type=FieldType.Float32,
        default_val=0
    ),
    Field(
        field_name="author",
        field_type=FieldType.List_String,
        default_val=[]
    ),
    Field(
        field_name="aim",
        field_type=FieldType.Bool,
        default_val=True
    ),
]
res = vikingdb_service.create_collection("examples", fields, "This is an example")
print(res)

# 图片数据读取

In [17]:
import csv
import sys

def process_csv(csv_filepath):
    """读取 CSV 文件，处理第四列 Base64 数据，并添加到 RawData 对象列表中。"""
    raw_data_list = []
    try:
        # 增加 CSV 字段大小限制
        maxInt = sys.maxsize
        while True:
            try:
                csv.field_size_limit(maxInt)
                break
            except OverflowError:
                maxInt = int(maxInt / 10)
        with open(csv_filepath, 'r', newline='', encoding='utf-8') as csvfile:
            reader = csv.reader(csvfile)
            next(reader)  # 跳过表头行 (如果存在)

            for row in reader:
                try:
                    base64_data = row[3]  # 获取第四列数据
                    # 创建 RawData 对象
                    raw_data_list.append(RawData("image", base64_data))
                except IndexError:
                    print(f"警告: 行 {reader.line_num} 数据不足，跳过。")
                except Exception as e:
                    print(f"错误: 处理行 {reader.line_num} 时出错: {e}")

        return raw_data_list
    except FileNotFoundError:
        print(f"错误: 文件 {csv_filepath} 未找到。")
        return None
    except Exception as e:
        print(f"错误: 读取或处理 CSV 文件时出错: {e}")
        return None



if __name__ == "__main__":
    csv_file_path = "base64_images.csv"  # 请替换为您的 CSV 文件路径
    raw_data_list = process_csv(csv_file_path)

    if raw_data_list:
        print("RawData 对象列表:")
        for raw_data in raw_data_list:
            print(raw_data)


RawData 对象列表:


# 图片向量化

In [18]:
res = vikingdb_service.embedding_v2(EmbModel("bge-m3", params={"return_token_usage": True}), raw_data_list)
print(res)


VikingDBException: json load res error, res:b'<html>\r\n<head><title>502 Bad Gateway</title></head>\r\n<body bgcolor="white">\r\n<center><h1>502 Bad Gateway</h1></center>\r\n</body>\r\n</html>\r\n'